In [3]:
import random
from numpy import pi
from qiskit import *
from qiskit.tools.visualization import plot_bloch_multivector

In [6]:
##### Simulación de funcionamiento de Protocolo BB84

# Definimos variables
bits_Alice = []
bases_Alice = []
bases_Bob = []

# Definimos la cantidad de bits a usar
num_bits = 16

# Función para transmitir Qubits
def transmitirQubits(bits, base):
    txQ = []
    for i in range(len(bits)):
        mycircuit = QuantumCircuit(1, 1)
        if bits[i] == '1':
            mycircuit.x(0)  # Aplicar la compuerta X para cambiar ket1 a |0⟩
        if base[i] == 'X':
            mycircuit.h(0)  # Aplicar la compuerta Hadamard para cambiar la base
        txQ.append(mycircuit)
    return txQ

# Función para medir (recibir) Qubits
def medirQubits(qubits, bases):
    rxQ = []
    for i in range(len(bases)):
        qubit = qubits[i]
        if bases[i] == 'Z':
            qubit.measure(0, 0)  # Medir en la base Z
        else:
            qubit.h(0)  # Aplicar la compuerta Hadamard para cambiar la base
            qubit.measure(0, 0)  # Medir en la base Hadamard
        job = execute(qubit, Aer.get_backend('qasm_simulator'), shots=1)
        result = job.result().get_counts(qubit)
        rxQ.append(list(result.keys())[0])  # Almacenar directamente el resultado de la medición
    return rxQ

# Lado de Alice: Generamos los bits aleatorios
for i in range(num_bits):
    bases_Alice.append(random.choice(['X', 'Z']))
    bits_Alice.append(random.choice(['0', '1']))
    bases_Bob.append(random.choice(['X', 'Z']))
    
tx_Alice = transmitirQubits(bits_Alice, bases_Alice) 

print("########## SIMULACION DE PROTOCOLO BB84 SIN PRESENCIA DE EVE ##########", "\n")

print("########## Transmisión de Qubits de Alice a Bob ##########", "\n")
print("Bases de Alice: ", bases_Alice,"\n")
print("Bits de Alice:  ", bits_Alice, "\n")

# Lado de Bob: Genera sus bases y mide los qubits enviados por Alice
rx_Bob = medirQubits(tx_Alice, bases_Bob)

print("########## Generación de Bases de Bob para medición ##########", "\n")
print("Bases de Bob:   ", bases_Bob, "\n")

# Se comparan las bases y se arma la clave con los Qbits coincidentes
clave = ""
aciertos = 0

# Comparación de Bases entre Alice y Bob
clave = ""
aciertos = 0

print("########## Comparación de Bases y Armado de Clave ##########", "\n")
print(f"{'Base Alice': <12} {'Base Bob': <12} {'Bit': <10} {'Resultado': <10}")

for i in range(num_bits):
    resultado = bits_Alice[i] if bases_Alice[i] == bases_Bob[i] else "-"
    if resultado != "-":
        clave += bits_Alice[i]
        aciertos += 1
    print(f"{bases_Alice[i]: <12} {bases_Bob[i]: <12} {bits_Alice[i]: <10} {resultado: <10}")

print("\nClave: ", clave)

porcentaje_aciertos = (aciertos / num_bits) * 100
print("Porcentaje de aciertos entre Alice y Bob:", porcentaje_aciertos, "%")

# Verificar consistencia entre bases y codificación
consistencia = all(resultado == bits_Alice[i] for i, resultado in enumerate(rx_Bob) if bases_Alice[i] == bases_Bob[i])
print("Porcentaje de consistencia entre bases y codificación:", "100%" if consistencia else "No hay consistencia", "\n")

########## SIMULACION DE PROTOCOLO BB84 SIN PRESENCIA DE EVE ########## 

########## Transmisión de Qubits de Alice a Bob ########## 

Bases de Alice:  ['Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z'] 

Bits de Alice:   ['0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '1', '1', '1'] 

########## Generación de Bases de Bob para medición ########## 

Bases de Bob:    ['Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X'] 

########## Comparación de Bases y Armado de Clave ########## 

Base Alice   Base Bob     Bit        Resultado 
Z            Z            0          0         
X            Z            0          -         
X            Z            1          -         
Z            X            0          -         
Z            X            1          -         
X            Z            0          -         
X            X            0          0         
Z            Z            0          0         
X    